In [2]:
with open('granny/imagenet_classes.txt', 'r') as f:
    categories = [s.strip() for s in f.readlines()]

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import numpy as np
from PIL import Image

In [4]:
# Load the pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.eval()

# torch.manual_seed(42)
use_cuda = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", device)

C:\Users\TTTTTT\PycharmProjects\kaggle_defcon_ai\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\TTTTTT\PycharmProjects\kaggle_defcon_ai\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\TTTTTT/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 33.7MB/s]


Device:  cpu


In [27]:
def run(image, padding=0, kernel_size=0, minimum=0, maximum=0):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Pad(padding),
        transforms.GaussianBlur(kernel_size, sigma=(minimum, maximum)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Assume `image` is a PIL image
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    # print(input_tensor.shape)
    # Perform inference
    with torch.no_grad():
        output = model(input_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    # print(probabilities)
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    return top5_prob, top5_catid
    

In [28]:
# Load the image
image = Image.open('granny/timber_wolf.png')

In [29]:
for padding in range(6):
    for kernel_size in range(1, 10, 2):
        for minimum in np.arange(0.1, 2, 0.1):
            for maximum in np.arange(minimum, 2, 0.1):
                
                top5_prob, top5_catid = run(image, padding, kernel_size, minimum, maximum)
                if categories[top5_catid[0]] == 'timber wolf' and (0.28 < top5_prob[0].item() < 0.29):
                    print(padding, kernel_size, minimum, maximum)
                    for i in range(top5_prob.size(0)):
                        print(categories[top5_catid[i]], top5_prob[i].item())

# run(image)

# 1 5 1.0 1.6
# 3 7 0.9 1.9

1 5 1.0 1.6000000000000005
timber wolf 0.2857857048511505
red wolf 0.14339323341846466
coyote 0.1347683072090149
dingo 0.08368808776140213
white wolf 0.0775667130947113
1 5 1.1 1.8000000000000007
timber wolf 0.2841856777667999
red wolf 0.14301109313964844
coyote 0.1352909803390503
dingo 0.08360335975885391
white wolf 0.07789673656225204
1 5 1.2000000000000002 1.8000000000000007
timber wolf 0.28768905997276306
red wolf 0.14378631114959717
coyote 0.13570016622543335
dingo 0.08307091891765594
white wolf 0.07789082825183868
1 7 0.2 1.8000000000000005
timber wolf 0.28457364439964294
red wolf 0.1446043848991394
coyote 0.1341276466846466
dingo 0.08642062544822693
white wolf 0.0829705223441124
1 7 0.6 1.4
timber wolf 0.28697341680526733
red wolf 0.14501458406448364
coyote 0.13339102268218994
dingo 0.08564907312393188
white wolf 0.08322794735431671
1 7 1.0 1.3000000000000003
timber wolf 0.28369200229644775
red wolf 0.14329928159713745
coyote 0.13536015152931213
dingo 0.08699982613325119
white w